In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
import numpy as np

In [2]:
print('hello')

hello


In [3]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.raw_data = dataframe
        self.labels = list(map(lambda x : [1] if x=='good' else [0], self.raw_data['Quality']))
        self.raw_data = self.raw_data.drop(labels='Quality', axis = 1)
        self.raw_data = self.raw_data.drop(labels='A_id', axis = 1)
        self.raw_data = self.raw_data.to_numpy()
        self.labels = np.array(self.labels)
        self.raw_data = torch.from_numpy(self.raw_data).float()
        self.labels = torch.from_numpy(self.labels).float()
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = self.raw_data[idx]
        label = self.labels[idx]
        if self.transform:
            item = self.transform(item)
        if self.target_transform:
            label = self.target_transform(label)
        sample = {'item' : item, 'label' : label}
        return item, label

In [4]:
def split_data(csv_data):
    splited_idx = int(len(csv_data) * 0.7)
    splitted_train = csv_data[:splited_idx]
    splitted_test = csv_data[splited_idx:]
    return splitted_train, splitted_test

In [5]:
csv_dir = '/home/jetson/pytorch/data/apple_quality.csv'

csv_data = pd.read_csv(csv_dir)
csv_train, csv_test = split_data(csv_data)

train_data = CustomDataset(csv_train)
test_data = CustomDataset(csv_test)

batch_size = 5

train_dataloader = DataLoader(dataset = train_data, batch_size=batch_size)
test_dataloader = DataLoader(dataset = test_data, batch_size=batch_size)

In [24]:
len(train_data)

2800

In [25]:
len(train_dataloader)

560

In [22]:
for batch, (X, y) in enumerate(train_dataloader):
    print(batch)
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

0
Shape of X [N, C, H, W]: torch.Size([5, 7])
Shape of y: torch.Size([5, 1]) torch.float32


In [6]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, 1)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
loss_fn = nn.BCELoss()
sigmoid = nn.Sigmoid()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        pred = model(x)        
        pred = sigmoid(pred)        
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) *len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print(f"size: {size}")
    num_batches = len(dataloader)
    print(f'num_batches: {num_batches}')
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad(): #-----> no_grad() gradient를 트래킹 하지않음. 메모리 사용량을 줄여 연산속도를 높이기 위해 사용됨.
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            pred = sigmoid(pred)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            print(test_loss)
            print(correct)
    print(f'final test_loss : {test_loss}')
    test_loss /= num_batches
    print(f'final correct : {correct}')
    correct /= size
    
    print(f"Test Error: \n Accuracy : {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


In [16]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    break
print("Done!")

Epoch 1
---------------------------
loss: 0.268261  [    5/ 2800]
loss: 0.336803  [  505/ 2800]
loss: 0.424600  [ 1005/ 2800]
loss: 0.469501  [ 1505/ 2800]
loss: 0.484003  [ 2005/ 2800]
loss: 0.429622  [ 2505/ 2800]
size: 1200
num_batches: 240
tensor(0.5530, device='cuda:0')
5.0
tensor(0.9194, device='cuda:0')
20.0
tensor(1.4150, device='cuda:0')
35.0
tensor(1.6912, device='cuda:0')
40.0
tensor(2.1767, device='cuda:0')
50.0
tensor(2.5905, device='cuda:0')
65.0
tensor(2.8770, device='cuda:0')
80.0
tensor(3.3784, device='cuda:0')
95.0
tensor(3.5928, device='cuda:0')
100.0
tensor(4.0436, device='cuda:0')
105.0
tensor(4.7155, device='cuda:0')
115.0
tensor(5.3093, device='cuda:0')
135.0
tensor(5.7245, device='cuda:0')
135.0
tensor(6.3380, device='cuda:0')
145.0
tensor(7.0246, device='cuda:0')
160.0
tensor(7.5055, device='cuda:0')
170.0
tensor(7.8816, device='cuda:0')
185.0
tensor(8.3519, device='cuda:0')
200.0
tensor(8.7991, device='cuda:0')
210.0
tensor(9.0995, device='cuda:0')
215.0
tenso

tensor(111.1287, device='cuda:0')
2505.0
tensor(111.6208, device='cuda:0')
2515.0
tensor(112.0880, device='cuda:0')
2530.0
tensor(112.5737, device='cuda:0')
2550.0
tensor(113.1767, device='cuda:0')
2560.0
tensor(113.7283, device='cuda:0')
2570.0
tensor(114.1446, device='cuda:0')
2580.0
tensor(114.5964, device='cuda:0')
2595.0
tensor(115.4059, device='cuda:0')
2615.0
tensor(115.9537, device='cuda:0')
2620.0
tensor(116.2570, device='cuda:0')
2635.0
tensor(116.9981, device='cuda:0')
2645.0
tensor(117.6427, device='cuda:0')
2660.0
tensor(117.9452, device='cuda:0')
2670.0
tensor(118.3255, device='cuda:0')
2685.0
tensor(118.6436, device='cuda:0')
2705.0
tensor(119.2495, device='cuda:0')
2730.0
tensor(119.7401, device='cuda:0')
2740.0
tensor(120.1133, device='cuda:0')
2755.0
tensor(120.6608, device='cuda:0')
2775.0
tensor(121.2302, device='cuda:0')
2780.0
tensor(121.7860, device='cuda:0')
2790.0
tensor(122.3639, device='cuda:0')
2795.0
tensor(122.6476, device='cuda:0')
2810.0
tensor(123.3378,